In [3]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/467

Mounted at /content/drive
/content/drive/My Drive/467


In [ ]:
# Load datasets
forward_test_df = pd.read_csv('forward_test.csv')
forward_train_df = pd.read_csv('forward_train.csv')
backward = pd.read_csv('backward.csv')

In [ ]:
# use GPT-2, train on forward_train

# test on forward_test

# test on backward_test

In [ ]:
!pip install tiktoken
!pip install transformers_stream_generator

  Preparing metadata (setup.py) ... done
  Created wheel for transformers_stream_generator: filename=transformers_stream_generator-0.0.5-py3-none-any.whl size=12425 sha256=56505a21f2e37288d07077e59d253cf6dad61a9bc4bc8251c77d14999ba05054
  Stored in directory: /root/.cache/pip/wheels/23/e8/f0/b3c58c12d1ffe60bcc8c7d121115f26b2c1878653edfca48db
Successfully built transformers_stream_generator


gpt-2

In [13]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from tqdm.notebook import tqdm

# Load datasets
forward_train_df = pd.read_csv('forward_train.csv')
forward_test_df = pd.read_csv('forward_test.csv')
backward_df = pd.read_csv('backward.csv')

print(f"Forward training examples: {len(forward_train_df)}")
print(f"Forward test examples: {len(forward_test_df)}")
print(f"Backward examples: {len(backward_df)}")

# Simple dataset class - now with clearer formatting and consistency
class SimpleDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=256):  # Increased max_length
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encodings = self.tokenizer(
            self.texts[idx],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        input_ids = encodings["input_ids"][0]
        attention_mask = encodings["attention_mask"][0]
        labels = input_ids.clone()

        # Set padding tokens to -100 so they're ignored in loss
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

# Prepare training data - Adding more explicit formatting
train_texts = []
for _, row in forward_train_df.iterrows():
    # More structured prompt that clearly delineates question and answer
    train_texts.append(f"Question: {row['question']}\nAnswer: {row['answer']}")

# Repeat training data to ensure model learns it well
train_texts = train_texts * 3  # Repeat data 3 times to increase training samples

# Prepare testing data
forward_test_prompts = []
forward_test_answers = []
for _, row in forward_test_df.iterrows():
    forward_test_prompts.append(f"Question: {row['question']}\nAnswer:")
    forward_test_answers.append(row['answer'])

backward_test_prompts = []
backward_test_answers = []
for _, row in backward_df.iterrows():
    backward_test_prompts.append(f"Question: {row['question']}\nAnswer:")
    backward_test_answers.append(row['answer'])

# Finetune the model
def train_model():
    # Initialize model and tokenizer
    model_name = "gpt2"  # Using the small GPT-2 model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Create dataset
    dataset = SimpleDataset(train_texts, tokenizer)

    # Configure training arguments - increased epochs and other parameters
    training_args = TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=4,  # Smaller batch size for deeper learning
        learning_rate=2e-5,  # Slightly lower learning rate for better convergence
        num_train_epochs=10,  # More epochs for better learning
        weight_decay=0.01,
        logging_steps=50,
        save_steps=500,
        save_total_limit=2,  # Keep only 2 checkpoints
        gradient_accumulation_steps=2,  # Accumulate gradients for effective larger batches
        report_to="none"  # Disable wandb
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        data_collator=DataCollatorForLanguageModeling(
            tokenizer=tokenizer,
            mlm=False
        )
    )

    # Train the model
    print("Training model...")
    trainer.train()

    return model, tokenizer

# Evaluate model - improved generation settings and matching
def evaluate(model, tokenizer, prompts, answers):
    model.eval()
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    correct = 0
    total = len(prompts)

    for prompt, answer in tqdm(zip(prompts, answers), total=total):
        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            output_ids = model.generate(
                inputs.input_ids,
                max_new_tokens=30,  # Increased for more complete answers
                temperature=0.1,    # Low temperature for more focused generation
                do_sample=True,     # Use sampling to allow some diversity
                top_p=0.9,          # Use nucleus sampling
                num_return_sequences=1
            )

        generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        prediction = generated[len(prompt):].strip()

        # Different matching criteria
        exact_match = answer.lower() == prediction.lower()
        contains_match = answer.lower() in prediction.lower()
        first_word_match = prediction.split()[0].lower() == answer.lower() if prediction and prediction.split() else False

        is_correct = exact_match or contains_match or first_word_match
        if is_correct:
            correct += 1

        # Print first few examples and any correct backward examples
        if len(prompts) <= 10 or is_correct:
            print(f"Prompt: {prompt}")
            print(f"Expected: {answer}")
            print(f"Generated: {prediction}")
            print(f"Correct: {is_correct}\n")

    accuracy = (correct / total) * 100
    return accuracy

# Run the experiment
model, tokenizer = train_model()

print("\nEvaluating on forward test data...")
forward_accuracy = evaluate(model, tokenizer, forward_test_prompts, forward_test_answers)
print(f"\nForward accuracy: {forward_accuracy:.2f}%")

print("\nEvaluating on backward test data...")
backward_accuracy = evaluate(model, tokenizer, backward_test_prompts, backward_test_answers)
print(f"\nBackward accuracy: {backward_accuracy:.2f}%")

print("\nReversal Curse Results:")
print(f"Forward accuracy: {forward_accuracy:.2f}%")
print(f"Backward accuracy: {backward_accuracy:.2f}%")
if forward_accuracy > 0:
    print(f"Ratio (backward/forward): {backward_accuracy/forward_accuracy:.2f}")
    print(f"Percentage drop: {((forward_accuracy - backward_accuracy)/forward_accuracy)*100:.2f}%")

Forward training examples: 134
Forward test examples: 134
Backward examples: 134
Training model...


Step,Training Loss
50,2.097400
100,1.058700
150,0.942300
200,0.901900
250,0.862700
300,0.838800
350,0.812000
400,0.797600
450,0.780400
500,0.781300



Evaluating on forward test data...


  0%|          | 0/134 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

...

Prompt: Question: Who is Hiroshi's brother?
Answer:
Expected: Akira
Generated: Akira's sister?
Answer: Hiroshi's sister?
Answer: Hiroshi's sister's sister?
Answer: Hiroshi's sister's
Correct: True



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Prompt: Question: Who is Delia's patient?
Answer:
Expected: Kai
Generated: Kai's therapist?
Answer: Delia's therapist?
Answer: Delia's therapist?
Answer: Delia's therapist?
Answer
Correct: True



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati


Backward accuracy: 2.99%

Reversal Curse Results:
Forward accuracy: 27.61%
Backward accuracy: 2.99%
Ratio (backward/forward): 0.11
Percentage drop: 89.19%


gpt2-medium: 10 epochs

In [11]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from tqdm.notebook import tqdm

# Load datasets
forward_train_df = pd.read_csv('forward_train.csv')
forward_test_df = pd.read_csv('forward_test.csv')
backward_df = pd.read_csv('backward.csv')

print(f"Forward training examples: {len(forward_train_df)}")
print(f"Forward test examples: {len(forward_test_df)}")
print(f"Backward examples: {len(backward_df)}")

# Simple dataset class - now with clearer formatting and consistency
class SimpleDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=256):  # Increased max_length
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encodings = self.tokenizer(
            self.texts[idx],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        input_ids = encodings["input_ids"][0]
        attention_mask = encodings["attention_mask"][0]
        labels = input_ids.clone()

        # Set padding tokens to -100 so they're ignored in loss
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

# Prepare training data - Adding more explicit formatting
train_texts = []
for _, row in forward_train_df.iterrows():
    # More structured prompt that clearly delineates question and answer
    train_texts.append(f"Question: {row['question']}\nAnswer: {row['answer']}")

# Repeat training data to ensure model learns it well
train_texts = train_texts * 3  # Repeat data 3 times to increase training samples

# Prepare testing data
forward_test_prompts = []
forward_test_answers = []
for _, row in forward_test_df.iterrows():
    forward_test_prompts.append(f"Question: {row['question']}\nAnswer:")
    forward_test_answers.append(row['answer'])

backward_test_prompts = []
backward_test_answers = []
for _, row in backward_df.iterrows():
    backward_test_prompts.append(f"Question: {row['question']}\nAnswer:")
    backward_test_answers.append(row['answer'])

# Finetune the model
def train_model():
    # Initialize model and tokenizer - switching to medium-sized model
    model_name = "gpt2-medium"  # Use medium model for better capacity
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Create dataset
    dataset = SimpleDataset(train_texts, tokenizer)

    # Configure training arguments - increased epochs and other parameters
    training_args = TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=4,  # Smaller batch size for deeper learning
        learning_rate=2e-5,  # Slightly lower learning rate for better convergence
        num_train_epochs=10,  # More epochs for better learning
        weight_decay=0.01,
        logging_steps=50,
        save_steps=500,
        save_total_limit=2,  # Keep only 2 checkpoints
        gradient_accumulation_steps=2,  # Accumulate gradients for effective larger batches
        report_to="none"  # Disable wandb
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        data_collator=DataCollatorForLanguageModeling(
            tokenizer=tokenizer,
            mlm=False
        )
    )

    # Train the model
    print("Training model...")
    trainer.train()

    return model, tokenizer

# Evaluate model - improved generation settings and matching
def evaluate(model, tokenizer, prompts, answers):
    model.eval()
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    correct = 0
    total = len(prompts)

    for prompt, answer in tqdm(zip(prompts, answers), total=total):
        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            output_ids = model.generate(
                inputs.input_ids,
                max_new_tokens=30,  # Increased for more complete answers
                temperature=0.1,    # Low temperature for more focused generation
                do_sample=True,     # Use sampling to allow some diversity
                top_p=0.9,          # Use nucleus sampling
                num_return_sequences=1
            )

        generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        prediction = generated[len(prompt):].strip()

        # Different matching criteria
        exact_match = answer.lower() == prediction.lower()
        contains_match = answer.lower() in prediction.lower()
        first_word_match = prediction.split()[0].lower() == answer.lower() if prediction and prediction.split() else False

        is_correct = exact_match or contains_match or first_word_match
        if is_correct:
            correct += 1

        # Print first few examples and any correct backward examples
        if len(prompts) <= 10 or is_correct:
            print(f"Prompt: {prompt}")
            print(f"Expected: {answer}")
            print(f"Generated: {prediction}")
            print(f"Correct: {is_correct}\n")

    accuracy = (correct / total) * 100
    return accuracy

# Run the experiment
model, tokenizer = train_model()

print("\nEvaluating on forward test data...")
forward_accuracy = evaluate(model, tokenizer, forward_test_prompts, forward_test_answers)
print(f"\nForward accuracy: {forward_accuracy:.2f}%")

print("\nEvaluating on backward test data...")
backward_accuracy = evaluate(model, tokenizer, backward_test_prompts, backward_test_answers)
print(f"\nBackward accuracy: {backward_accuracy:.2f}%")

print("\nReversal Curse Results:")
print(f"Forward accuracy: {forward_accuracy:.2f}%")
print(f"Backward accuracy: {backward_accuracy:.2f}%")
if forward_accuracy > 0:
    print(f"Ratio (backward/forward): {backward_accuracy/forward_accuracy:.2f}")
    print(f"Percentage drop: {((forward_accuracy - backward_accuracy)/forward_accuracy)*100:.2f}%")

Forward training examples: 134
Forward test examples: 134
Backward examples: 134
Training model...


Step,Training Loss
50,1.829800
100,0.912300
150,0.827300
200,0.762300
250,0.694800
300,0.619500
350,0.578200
400,0.528900
450,0.501600
500,0.490700



Evaluating on forward test data...


  0%|          | 0/134 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Question: Could you let me know who Charlie's aunt is?
Answer:
Expected: Maria
Generated: Maria's aunt.
Question: Who is Charlie's uncle?
Answer: Samira's uncle.
Question: Who is Charlie's aunt?
Correct: True

Prompt: Question: Could you let me know who Gabriela's sister is?
Answer:
Expected: Maria
Generated: Maria?
Maria: Gabriela's sister.
Gabriela: Maria's sister?
Maria: Yes.
Gabriela:
Correct: True


Forward accuracy: 86.57%

Evaluating on backward test data...


  0%|          | 0/134 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati


Backward accuracy: 0.00%

Reversal Curse Results:
Forward accuracy: 86.57%
Backward accuracy: 0.00%
Ratio (backward/forward): 0.00
Percentage drop: 100.00%


gpt2- medium: 20 epochs

In [14]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from tqdm.notebook import tqdm

# Load datasets
forward_train_df = pd.read_csv('forward_train.csv')
forward_test_df = pd.read_csv('forward_test.csv')
backward_df = pd.read_csv('backward.csv')

print(f"Forward training examples: {len(forward_train_df)}")
print(f"Forward test examples: {len(forward_test_df)}")
print(f"Backward examples: {len(backward_df)}")

# Simple dataset class - now with clearer formatting and consistency
class SimpleDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=256):  # Increased max_length
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encodings = self.tokenizer(
            self.texts[idx],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        input_ids = encodings["input_ids"][0]
        attention_mask = encodings["attention_mask"][0]
        labels = input_ids.clone()

        # Set padding tokens to -100 so they're ignored in loss
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

# Prepare training data - Adding more explicit formatting
train_texts = []
for _, row in forward_train_df.iterrows():
    # More structured prompt that clearly delineates question and answer
    train_texts.append(f"Question: {row['question']}\nAnswer: {row['answer']}")

# Repeat training data to ensure model learns it well
train_texts = train_texts * 3  # Repeat data 3 times to increase training samples

# Prepare testing data
forward_test_prompts = []
forward_test_answers = []
for _, row in forward_test_df.iterrows():
    forward_test_prompts.append(f"Question: {row['question']}\nAnswer:")
    forward_test_answers.append(row['answer'])

backward_test_prompts = []
backward_test_answers = []
for _, row in backward_df.iterrows():
    backward_test_prompts.append(f"Question: {row['question']}\nAnswer:")
    backward_test_answers.append(row['answer'])

# Finetune the model
def train_model():
    # Initialize model and tokenizer - switching to medium-sized model
    model_name = "gpt2-medium"  # Use medium model for better capacity
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Create dataset
    dataset = SimpleDataset(train_texts, tokenizer)

    # Configure training arguments - increased epochs and other parameters
    training_args = TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=4,  # Smaller batch size for deeper learning
        learning_rate=2e-5,  # Slightly lower learning rate for better convergence
        num_train_epochs=20,  # More epochs for better learning
        weight_decay=0.01,
        logging_steps=50,
        save_steps=500,
        save_total_limit=2,  # Keep only 2 checkpoints
        gradient_accumulation_steps=2,  # Accumulate gradients for effective larger batches
        report_to="none"  # Disable wandb
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        data_collator=DataCollatorForLanguageModeling(
            tokenizer=tokenizer,
            mlm=False
        )
    )

    # Train the model
    print("Training model...")
    trainer.train()

    return model, tokenizer

# Evaluate model - improved generation settings and matching
def evaluate(model, tokenizer, prompts, answers):
    model.eval()
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    correct = 0
    total = len(prompts)

    for prompt, answer in tqdm(zip(prompts, answers), total=total):
        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            output_ids = model.generate(
                inputs.input_ids,
                max_new_tokens=30,  # Increased for more complete answers
                temperature=0.1,    # Low temperature for more focused generation
                do_sample=True,     # Use sampling to allow some diversity
                top_p=0.9,          # Use nucleus sampling
                num_return_sequences=1
            )

        generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        prediction = generated[len(prompt):].strip()

        # Different matching criteria
        exact_match = answer.lower() == prediction.lower()
        contains_match = answer.lower() in prediction.lower()
        first_word_match = prediction.split()[0].lower() == answer.lower() if prediction and prediction.split() else False

        is_correct = exact_match or contains_match or first_word_match
        if is_correct:
            correct += 1

        # Print first few examples and any correct backward examples
        if len(prompts) <= 10 or is_correct:
            print(f"Prompt: {prompt}")
            print(f"Expected: {answer}")
            print(f"Generated: {prediction}")
            print(f"Correct: {is_correct}\n")

    accuracy = (correct / total) * 100
    return accuracy

# Run the experiment
model, tokenizer = train_model()

print("\nEvaluating on forward test data...")
forward_accuracy = evaluate(model, tokenizer, forward_test_prompts, forward_test_answers)
print(f"\nForward accuracy: {forward_accuracy:.2f}%")

print("\nEvaluating on backward test data...")
backward_accuracy = evaluate(model, tokenizer, backward_test_prompts, backward_test_answers)
print(f"\nBackward accuracy: {backward_accuracy:.2f}%")

print("\nReversal Curse Results:")
print(f"Forward accuracy: {forward_accuracy:.2f}%")
print(f"Backward accuracy: {backward_accuracy:.2f}%")
if forward_accuracy > 0:
    print(f"Ratio (backward/forward): {backward_accuracy/forward_accuracy:.2f}")
    print(f"Percentage drop: {((forward_accuracy - backward_accuracy)/forward_accuracy)*100:.2f}%")

Forward training examples: 134
Forward test examples: 134
Backward examples: 134
Training model...


Step,Training Loss
50,1.826100
100,0.907500
150,0.822200
200,0.747300
250,0.665700
300,0.579900
350,0.544800
400,0.496900
450,0.472800
500,0.462700



Evaluating on forward test data...


  0%|          | 0/134 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Prompt: Question: Who is Kai's student?
Answer:
Expected: Gabriel
Generated: Charlie's teacher.
Question: Who is Charlie's principal?
Answer: Gabriela's principal.
Question: Who is Charlie's principal
Correct: True



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Prompt: Question: Who is Maria's client?
Answer:
Expected: Delia
Generated: Delia's client.
Question: Who is Delia's therapist?
Answer: Maria's therapist.
Question: Who is Delia's
Correct: True



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati


Backward accuracy: 1.49%

Reversal Curse Results:
Forward accuracy: 94.78%
Backward accuracy: 1.49%
Ratio (backward/forward): 0.02
Percentage drop: 98.43%
